# gensim

Poznamy teraz inną bibliotekę do przetwarzania tekstu - gensim. 

Jest bardziej wydajna od nltk, pozwala przetwarzać duże zbiory dokumentów. 

Trzeba zainstalować gensim: https://radimrehurek.com/gensim/install.html


# Tworzenie reprezentacji wektorowej

Spróbujmy wykonać w gensim to czego się już nauczyliśmy, czyli transformować korpus do postaci wektorowej.

In [17]:
from gensim import corpora
import numpy as np

Tworzymy sobie zbiór dokumentów.

In [20]:
documents = ["Romeo and Juliet",
         "Juliet: O happy dagger",
         "Romeo died by dagger",
         "'Live free or die', that’s the New-Hampshire’s motto",
         "Did you know, New-Hampshire is in New-England"]

np.savetxt('mycorpus.txt', documents, fmt='%s')

# Zad

Oczyszczamy tekst:

* tokenizacja

* usuwanie stopwords

* usuwanie tokenów występujących tylko 1 raz

* zmiana dużych liter na małe

Do powyższych zadań gensim nie ma dedykowanych narzędzi.

In [21]:
from pprint import pprint  # pretty-printer
from collections import defaultdict

# remove common words and tokenize
stoplist = set('for a of the and to in'.split())
texts = [[word for word in document.lower().split() if word not in stoplist]
          for document in documents]

# # remove words that appear only once
# frequency = defaultdict(int)
# for text in texts:
#     for token in text:
#         frequency[token] += 1

# texts = [[token for token in text if frequency[token] > 1]
#          for text in texts]

pprint(texts)

[['romeo', 'juliet'],
 ['juliet:', 'o', 'happy', 'dagger'],
 ['romeo', 'died', 'by', 'dagger'],
 ["'live", 'free', 'or', "die',", 'that’s', 'new-hampshire’s', 'motto'],
 ['did', 'you', 'know,', 'new-hampshire', 'is', 'new-england']]


# Chcemy utworzyć reprezentację bag-of-words.

* Pakiet corpora posiada klasę Dictionary, która tworzy słownik z zadanych tokenów. 

* Odpowiada to sklearnowemu fit. 

* Słownik można zapisać na dysku i później go odzyskać.

In [22]:
dictionary = corpora.Dictionary(texts)
dictionary.save('tmp/deerwester.dict')  # store the dictionary, for future reference
print(dictionary)

Dictionary(21 unique tokens: ['juliet', 'romeo', 'dagger', 'happy', 'juliet:']...)


Kluczami są tokeny a wartościami kolejne identyfikatory.

In [23]:
print(dictionary.token2id)

{'juliet': 0, 'romeo': 1, 'dagger': 2, 'happy': 3, 'juliet:': 4, 'o': 5, 'by': 6, 'died': 7, "'live": 8, "die',": 9, 'free': 10, 'motto': 11, 'new-hampshire’s': 12, 'or': 13, 'that’s': 14, 'did': 15, 'is': 16, 'know,': 17, 'new-england': 18, 'new-hampshire': 19, 'you': 20}


* Mając słownik możemy wziąć dowolny nowy dokument i przetransformować go. 

* To jest sklearnowy transform.

In [25]:
new_doc =  "died dagger"
new_vec = dictionary.doc2bow(new_doc.lower().split())
print(new_vec) 

[(2, 1), (7, 1)]


Funkcja doc2bow zlicza częstości wystąpienia kolejnych tokenów słownika w dokumencie. Zwraca wynik jako sparse vector.

# Zad
Prosze przetransformować "texts" do postaci wektorowej (nie ma do tego dedykowanej funkcji - trzeba użyć podejścia pythonowego)

In [7]:
corpus = [dictionary.doc2bow(text) for text in texts]
pprint(corpus)

corpus = [dictionary.doc2bow(text) for text in texts]
print(corpus)

[[(0, 1), (1, 1)],
 [(2, 1), (3, 1), (4, 1), (5, 1)],
 [(1, 1), (2, 1), (6, 1), (7, 1)],
 [(8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1)],
 [(15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1)]]
[[(0, 1), (1, 1)], [(2, 1), (3, 1), (4, 1), (5, 1)], [(1, 1), (2, 1), (6, 1), (7, 1)], [(8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1)], [(15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1)]]


Tą reprezentację również można zapisać na dysku.

In [8]:
corpora.MmCorpus.serialize('tmp/deerwester.mm', corpus)  # store to disk, for later use

Korpusem w gensim jest macierz rzadka.

# Efektywne przetwarzanie tekstu

gensim pozwala operować na dużych zbiorach tekstu. Do pamięci nie jest wczytywany cały korpus a jedynie dokument po dokumencie.

Tworzymy klasę korpusu z funkcę __iter__, która ma dostęp do kolejnych tokenów korpusu. 

Postać tej funkcji może zostać dopasowana do jakiegokolwiek formatu naszego korpusu.

In [9]:
class MyCorpus(object):
    def __iter__(self):
        for line in open('mycorpus.txt'):
            # assume there's one document per line, tokens separated by whitespace
            yield dictionary.doc2bow(line.lower().split())

* Możemy stowrzyć obiekt korpusu. 

* Nie wczytuje to całego korpusu do pamięci. 

* W danym momencie tylko jeden dokument jest w pamięci.

In [10]:
corpus_memory_friendly = MyCorpus()  # doesn't load the corpus into memory!
print(corpus_memory_friendly)

Żeby zobaczyć co się kryje pod korpusem iterujemy po wektorach

In [11]:
for vector in corpus_memory_friendly:  # load one vector into memory at a time
    print(vector)

[(0, 1), (1, 1)]
[(2, 1), (3, 1), (4, 1), (5, 1)]
[(1, 1), (2, 1), (6, 1), (7, 1)]
[(8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1)]
[(15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1)]


# Mimo że wynik nie różni się niczym od poprzedniego, to jest pamięciowo bardziej wydajny.

W podobny sposób możemy stworzyć słownik

In [12]:
from six import iteritems
# collect statistics about all tokens
dictionary = corpora.Dictionary(line.lower().split() for line in open('mycorpus.txt'))
# remove stop words and words that appear only once
stop_ids = [dictionary.token2id[stopword] for stopword in stoplist
            if stopword in dictionary.token2id]
once_ids = [tokenid for tokenid, docfreq in iteritems(dictionary.dfs) if docfreq == 1]
dictionary.filter_tokens(stop_ids + once_ids)  # remove stop words and words that appear only once
dictionary.compactify()  # remove gaps in id sequence after words that were removed
print(dictionary)


Dictionary(2 unique tokens: ['romeo', 'dagger'])


# Kompatybilność z numpy

Możemy transformować zarówno gęste jak i rzadkie macierze

In [13]:
import gensim
import numpy as np
numpy_matrix = np.random.randint(10, size=[5,2])  # random matrix as an example
print(numpy_matrix)
corpus = gensim.matutils.Dense2Corpus(numpy_matrix)

numpy_matrix = gensim.matutils.corpus2dense(corpus, num_terms=5)
print(numpy_matrix)

[[5 9]
 [4 1]
 [2 9]
 [5 3]
 [9 2]]
[[5. 9.]
 [4. 1.]
 [2. 9.]
 [5. 3.]
 [9. 2.]]


In [16]:
import scipy.sparse
scipy_sparse_matrix = scipy.sparse.random(5,2)  # random sparse matrix as example
#print(scipy_sparse_matrix.todense())
corpus = gensim.matutils.Sparse2Corpus(scipy_sparse_matrix)
#[pprint(text) for text in corpus]
scipy_csc_matrix = gensim.matutils.corpus2csc(corpus)
#print(scipy_csc_matrix.todense())